# ニューラルネットワークによる特徴生成の可視化
２クラス識別問題をニューラルネットワークで解く。最終層が線形ロジスティック回帰を実現し、最終層に至る複数の層が特徴抽出する。シンプルな２クラス識別問題で、特徴空間への写像の様子を観察する。

# 1. モジュールの読み込み
セル左上の三角形をクリックして実行してください。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import array
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Activation, Input
from keras import optimizers

# 2.1 学習データと性能評価用データの生成
以前の講義で利用したデータを用いることにする<p>
セル左上の三角形をクリックして実行してください。

In [ ]:
# 学習データの生成
N = 100
# クラス0はm0中心の正規分布
m0 = [0,0]
# クラス1はm1とm2中心の混合正規分布
m1 = [-4,4]
m2 = [4,-4]
# どの正規分布も共分散は共通で
Sigma = [[2,1],[1,2]]
np.random.seed( seed = 20)
# クラス0とクラス1のデータをぞれぞれN個ずつ生成
N=100
# x_0: 2次元の点がN個。y_0：それぞれの点のラベルはy=0
x_0 = np.random.multivariate_normal( m0, Sigma, N)
y_0 = np.zeros( len(x_0))
# クラス1のデータをN個生成
# x_1: 2次元の点がN個。y_1：それぞれの点のラベルはy=1
x_11 = np.random.multivariate_normal( m1, Sigma, (int)(N/2))
x_12 = np.random.multivariate_normal( m2, Sigma, (int)(N/2))
x_1 = np.concatenate( [x_11,x_12], axis=0 )
y_1 = np.ones( len( x_1 ))
# 双方のクラスをまとめる
x_train = np.concatenate([x_0, x_1], axis=0)
y_train = np.concatenate([y_0, y_1], axis=0)

# 評価用のデータも生成する
# x_0: 2次元の点がN個。y_0：それぞれの点のラベルはy=0
test_x_0 = np.random.multivariate_normal( m0, Sigma, N)
test_y_0 = np.zeros( len(test_x_0))
# クラス1のデータをN個生成
# x_1: 2次元の点がN個。y_1：それぞれの点のラベルはy=1
test_x_11 = np.random.multivariate_normal( m1, Sigma, (int)(N/2))
test_x_12 = np.random.multivariate_normal( m2, Sigma, (int)(N/2))
test_x_1 = np.concatenate( [test_x_11,test_x_12], axis=0 )
test_y_1 = np.ones( len( test_x_1 ))
# 双方のクラスをまとめる
x_test = np.concatenate([test_x_0, test_x_1], axis=0)
y_test = np.concatenate([test_y_0, test_y_1], axis=0)

# 2.2 データ分布の可視化
シンプルな線形識別では識別出来ない。適切な特徴空間への写像が必要。<p>
セル左上の三角形をクリックして実行してください。

In [ ]:
# 生成したデータの表示
plt.scatter( x_0[:,0], x_0[:,1])
plt.scatter( x_1[:,0], x_1[:,1])

# 3. ニューラルネットワークの構築
4層ニューラルネットワークを構築する。
* 1層目：ニューロンは２つ。x座標とy座標をそれぞれに入力。
* 2層目：ニューロンはZ_NUM個。活性化関数はReLU。
* 3層目：ニューロンは（可視化しやすいように）2個。活性化関数はReLU
* 4層目：最終層。ニューロンは1個。活性化関数はsigmoid関数。事後確率を回帰する。

三角形をクリックし、セルを上から順に実行してください。
* コスト関数は交差エントロピー関数（２クラスバージョン）
* 最適化にはsgdを使う

In [ ]:
# 2層目のニューロンの数
Z_NUM = 3

In [ ]:
# 層の構築
model = Sequential()
# 2層目の追加
model.add(Dense(Z_NUM, activation = 'relu', input_dim = 2))
# 3層目の追加（可視化しやすいように2個のニューロン)
model.add(Dense(2, activation = 'relu', input_dim = 2))
# 最終層の追加
model.add(Dense(1, activation = 'sigmoid'))
# コスト関数(loss関数)と最適化法の指定
model.compile( loss = 'binary_crossentropy', optimizer = 'sgd' )

# 3.1 ニューラルネットワークのパラメータの数
**model.summary()** により構築したニューラルネットワークの概要を表示できる。上が入力側・下が出力側。Output Shapeの列を見ると各層のニューロンの数が分かる。
バイアス項のための重みパラメータを忘れないように数える。
* 第2層への結線：(入力(2)＋バイアス項(1))*Z_NUM
* 第3層への結線：(Z_NUM + バイアス項(1))*第3層のニューロンの数(2)
* 第4層への結線： (第3層のニューロンの数(2) + バイアス項(1))*第4層のニューロンの数(1)

こんなに小さなニューラルネットワークでもパラメータ数が数十ある。

In [ ]:
# ニューラルネットワークの構成の表示
model.summary()

# 4. 学習
下記セルを上から順に実行してください。EPOCH数1000のとき2〜3分ほどかかります。

In [ ]:
# 更新回数
EPOCH = 1000
# バッチの大きさ
BATCH_SIZE = 5

In [ ]:
###### 学習開始
result = model.fit(x_train, y_train, epochs=EPOCH, batch_size = BATCH_SIZE)
# score = model.evaluate(x_test, y_test, batch_size=16)
print(result)

#4.1 学習過程の確認
コスト関数が更新とともにどのように小さくなっていったかを確認する。

In [ ]:
# エポック数とコストのグラフ
plt.plot(range(1, EPOCH+1), result.history['loss'], label="LOSS")
plt.xlabel('Epochs')
plt.ylabel('LOSS')
plt.legend()
plt.show()

# 4.2 正答率の確認
学習したニューラルネットワークが初見の評価用データをどの程度正確に識別するか確認する
* **x_test** 評価用のデータ。2.1で生成した。学習データには含まれていない。
* **model.predict** 学習済みのニューラルネットワークによる出力を計算
* **y_result** ニューラルネットワークの出力（y=1となる事後確率）を四捨五入することで0 or 1の出力に翻訳したもの
* 正解との比較により正答率を計算
  * **diff** 正解と予測との差分
  * **count_nonzero(diff==0)** 差分がゼロ（正解）の個数

In [ ]:
# 学習済みのニューラルネットワークに新しいデータを入れて 0 or 1を出力する
y_result = np.array([ float(i) for i in np.round(model.predict( x_test ))])
diff = np.array( [int(i) for i in y_result - y_test] )
print('正答率: ', np.count_nonzero( diff == 0 )/(N*2)*100, '%')

# 5 中間層の出力の可視化
最終層の直前（第3層）の値を確認する
* names[0], names[1], ...に、順に第2層、第3層、...の名前を格納する。
* **keras.Model:** 構築したニューラルネットワーク(model)から入出力層を抽出する

In [ ]:
# ニューラルネットワークの各層の名前を格納する
names = [l.name for l in model.layers]
# new_modelとして第3層（names[1])の出力をそのまま出すニューラルネットワークを新たに構築する
new_model = keras.Model(inputs=model.input, outputs=model.get_layer(names[1]).output)

# 5.1 中間層の出力の取得
* 学習用データを新規に構築したニューラルネットワーク(new_model)に入力
* 出力を new_outputに格納

**学習に成功していれば、クラス0とクラス1は線形分離出来るように分布するはず。**

In [ ]:
# クラス0のデータの第3層での値
z_0 = new_model(x_0)
# クラス1のデータの第3層での値
z_1 = new_model(x_1)
# グラフに分布をプロットする
plt.scatter( z_0[:,0], z_0[:,1])
plt.scatter( z_1[:,0], z_1[:,1])

#6. 最終的に得られた識別境界の確認
元の空間がどのように分割されたかを可視化する。

**学習に成功していれば、クラス0とクラス1の間に境界線が引かれるはず。**

In [ ]:
# 描画領域の確定
xmin = np.amin( x_train[:,0])
xmax = np.amax( x_train[:,0])
ymin = np.amin( x_train[:,1])
ymax = np.amax( x_train[:,1])
# 領域内にメッシュ生成
u = np.arange( xmin, xmax, 0.1)
v = np.arange( ymin, ymax, 0.1)
U, V = np.meshgrid( u, v )
# ニューラルネットワークに入力するために型変形
u_flatten = U.flatten()
v_flatten = V.flatten()
uv_flatten= np.stack([u_flatten, v_flatten], 1)
# 領域内に生成したメッシュ点それぞれの座標をニューラルネットワークに入力して出力計算
all_predict = (model.predict( uv_flatten )).reshape(U.shape)
# 識別境界（事後確率の回帰値が0.5)を描画する
plt.scatter( x_0[:,0], x_0[:,1])
plt.scatter( x_1[:,0], x_1[:,1])
nn_predict = plt.contour(U, V, all_predict, levels=[0.5])
nn_predict.clabel(fmt='%1.2f')